In [ ]:
import os
from pathlib import Path
from typing import List

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
# Plot distribution of embedding lenghts
fig, axs = plt.subplots(1, 2, figsize=(12, 5))
axs = np.ravel(axs)  # Flatten to 1D array
axs: List[plt.Axes]  # Type hinting

# Plot Arcface distribution
embed_arc = np.array(np.load("../results/xqlfw_arcface-r50.npy"))
print("Arcface matrix shape:", embed_arc.shape)
ampl_arc = np.linalg.norm(embed_arc, ord=2, axis=1)
axs[0].hist(ampl_arc, bins=50, edgecolor="black")
axs[0].set_xlim([10.0, 40.0])
axs[0].set_title("Arcface")

# Plot Magface distribution
embed_mag = np.array(np.load("../results/xqlfw_magface-r50.npy"))
print("Magface matrix shape:", embed_mag.shape)
ampl_mag = np.linalg.norm(embed_mag, ord=2, axis=1)
axs[1].hist(ampl_mag, bins=50, edgecolor="black")
axs[1].set_xlim([10.0, 40.0])
axs[1].set_title("Magface")

plt.show()

In [ ]:
# Plot images with lowest and highest MagFace embedding magnitude
img_list = []
with open("../results/xqlfw.txt", "r") as f:
    img_list = [line.rstrip("\n") for line in f]
# print(img_list[0])

data_p = Path(os.environ["DATASET_DIR"]) / "EvalDatasets" / "xqlfw" / "xqlfw_aligned_112"
# img_files = list(data_p.glob("*.jpg"))
# print(img_files[0])

# Sort magface embeddings by magnitude
sort_idx = np.argsort(ampl_mag)


def plot_images(axs: List[plt.Axes], order: str = "lowest"):
    for i in range(len(axs)):
        if order == "lowest":
            idx = sort_idx[i]
        elif order == "highest":
            idx = sort_idx[len(sort_idx) - 1 - i]
        img_name = img_list[idx]
        img = Image.open(data_p / img_name)
        axs[i].imshow(img)
        axs[i].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
        axs[i].set_xlabel(f"{ampl_mag[idx]:.3f}")


fig1, axs1 = plt.subplots(4, 4)
plot_images(axs=np.ravel(axs1), order="lowest")
fig1.tight_layout()
fig1.savefig("../results/magface_lowest.png")
plt.close(fig1)
fig2, axs2 = plt.subplots(4, 4)
plot_images(axs=np.ravel(axs2), order="highest")
fig2.tight_layout()
fig2.savefig("../results/magface_highest.png")
plt.close(fig2)


fig, axs = plt.subplots(1, 2, figsize=(15, 20))
axs: List[plt.Axes]  # Type hinting
axs[0].imshow(plt.imread("../results/magface_lowest.png"))
axs[0].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
axs[0].set_frame_on(False)
axs[0].set_title("Images with lowest magnitude")
axs[1].imshow(plt.imread("../results/magface_highest.png"))
axs[1].tick_params(left=False, bottom=False, labelleft=False, labelbottom=False)
axs[1].set_frame_on(False)
axs[1].set_title("Images with highest magnitude")
plt.show()